<a href="https://colab.research.google.com/github/ivan-pirner/ml/blob/autogluon/autoGluon_RG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# prompt: install autogluon

!pip install autogluon



In [ ]:
import warnings
import os
from datetime import datetime

from autogluon.multimodal import MultiModalPredictor
from ray import tune
import pandas as pd


warnings.filterwarnings('ignore')

In [ ]:
from autogluon.core.utils.loaders import load_zip

zip_file_url = "https://storage.googleapis.com/kaggle-competitions-data/kaggle-v2/82048/8963053/bundle/archive.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1737890592&Signature=mDLAw4XOG79AuUcTlCwq88vFjl77TRj%2FTToXQ4FnmEW9f10UUg8uoMhGTsiMct2tlQx7mZ0QItYyElnGyi8%2Fr6Le45AU7%2BNS5XO6wkpOOpwaNZ6NNpons016i7KAG37xXoU1xDPmCb3OjNK1%2BK4jga6Zc3Bvvb728udBFXrpLJ%2BkHYfFHHJ2JTrBRNHHPj%2FclFcTWAHv4l4OIL32n1Nce5iCxRcsh7%2FN1Vy07qBC0BvBp4r%2BM3nRcKsQ%2FY9rH3qiU9B7HjUQPZj5zbofaDHJsw5LzVFs4eEzdDP9hd%2BfMGsSvSSbHsjrxDSbfVFuins%2B4tAEzCZmtqEfvwhEmP%2BpTg%3D%3D&response-content-disposition=attachment%3B+filename%3Dne-mo-2-summer-challenge.zip"
download_dir = "./nemo2"

load_zip.unzip(zip_file_url, unzip_dir=download_dir)





In [ ]:
# prompt: find all jpeg files in /content/nemo2/train and put them into a pandas dataframe. Each file has a label according to its parrent folder named birdX, where X stands for the integer label

import pandas as pd
import os

def create_dataframe_from_images(root_dir):
    image_data = []
    for label_dir in os.listdir(root_dir):
        label_path = os.path.join(root_dir, label_dir)
        if os.path.isdir(label_path):
            try:
                label = int(label_dir.replace("bird", ""))
            except ValueError:
                continue # Skip directories that don't follow the birdX naming convention
            for filename in os.listdir(label_path):
                if filename.lower().endswith(('.jpg', '.jpeg')):
                    image_path = os.path.join(label_path, filename)
                    image_data.append({'image_path': image_path, 'label': label})
    return pd.DataFrame(image_data)

# Example usage
train_data = create_dataframe_from_images('/content/nemo2/train/spectrograms')
print(train_data.head())

In [ ]:
# prompt: launch tensorboard in a window

%load_ext tensorboard
%tensorboard --logdir /content/AutogluonModels


In [ ]:
predictor = MultiModalPredictor(
    label='label',
    presets="high_quality_hpo",
    )
predictor.fit(
    train_data=train_data,
    time_limit=1*60, # 1 minute
    # config=config,
    # hyperparameters=hyperparameters,
    # hyperparameter_tune_kwargs=hyperparameter_tune_kwargs,
    # seed=1
    )
